<a href="https://colab.research.google.com/github/a2m-dotcom/DLBCL_Pub/blob/main/TiL%20Waveform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
Minimal single WSI pipeline for methodology figure
Input: folder containing cells.geojson
Output: images + summary JSON
"""

from pathlib import Path
import json, math
import numpy as np
from collections import Counter, defaultdict
from PIL import Image, ImageDraw
from scipy.ndimage import gaussian_filter, binary_dilation, label, distance_transform_edt, binary_erosion
import matplotlib.pyplot as plt

# ---------- CONFIG ----------
WSI_FOLDER = Path("/content/drivee/MyDrive/DLBCLMORPH/Results/13980_1/13980_1")  # <- change here if needed
OUT_DIR = Path("/content/drivee/MyDrive/DLBCLMORPH/Results/13980_1/method_fig_outputs_single")
OUT_DIR.mkdir(parents=True, exist_ok=True)

downscale = 100
gaussian_sigma = 1.0
dilation_iters = 10
min_component_size = 100
zone2_dilation_size = 50

NUM_BINS_INSIDE = 25
NUM_BINS_OUTSIDE = 25
BIN_WIDTH_PIXELS = 2

CLASS_COLORS = {
    "Tumor": (220, 20, 60),
    "nonTIL Stromal": (210, 210, 180),
    "sTIL": (80, 160, 120),
    "Other": (160, 160, 200)
}

# ---------- Helpers ----------
def to_mask_coord(x, y, downscale):
    return int(math.floor(float(x)/downscale)), int(math.floor(float(y)/downscale))

def centroid(ring):
    xs = [p[0] for p in ring if p]
    ys = [p[1] for p in ring if p]
    return (sum(xs)/len(xs), sum(ys)/len(ys)) if xs else (None,None)

# ---------- Main ----------
geojson_path = WSI_FOLDER / "cells.geojson"
if not geojson_path.exists():
    raise FileNotFoundError(f"No geojson at {geojson_path}")

with open(geojson_path) as f:
    gj = json.load(f)
features = gj if isinstance(gj, list) else gj.get("features", [])
if not features:
    raise ValueError("Empty geojson")

# collect polygons
classes = defaultdict(list)
max_x=max_y=0
for feat in features:
    props = feat.get("properties", {}) or {}
    cname = props.get("classification", props.get("class"))
    if isinstance(cname, dict):
        cname = cname.get("name","Unknown")
    geom = feat.get("geometry",{})
    coords = geom.get("coordinates") or []
    if geom.get("type")=="Polygon":
        polys=[coords]
    elif geom.get("type")=="MultiPolygon":
        polys=coords
    else: polys=[]
    classes[cname].extend(polys)
    for poly in polys:
        for ring in poly:
            for x,y in ring:
                if x and y:
                    max_x=max(max_x,float(x)); max_y=max(max_y,float(y))

W,H=int(max_x//downscale)+2,int(max_y//downscale)+2

# rasterize tumor mask
tumor_polys = classes.get("Tumor", [])
mask_img = Image.new("L",(W,H),0)
d=ImageDraw.Draw(mask_img)
for poly in tumor_polys:
    if not poly: continue
    pts=[to_mask_coord(x,y,downscale) for x,y in poly[0]]
    d.polygon(pts,fill=1)
tumor_mask = np.array(mask_img)

# postprocess
from scipy.ndimage import label as cc_label
smooth = gaussian_filter(tumor_mask.astype(float),sigma=gaussian_sigma)
smooth=(smooth>0.5).astype(np.uint8)
dil=binary_dilation(smooth,iterations=dilation_iters).astype(np.uint8)
lab,_=cc_label(dil)
for l in range(1,_+1):
    if (lab==l).sum()<min_component_size:
        dil[lab==l]=0
tumor_mask_proc=dil

# zones
zone2 = binary_dilation(tumor_mask_proc,structure=np.ones((zone2_dilation_size,zone2_dilation_size))).astype(np.uint8)-tumor_mask_proc
zone3 = ((1-np.clip(tumor_mask_proc+zone2,0,1))).astype(np.uint8)

# save overlay
base=Image.new("RGBA",(W,H),(255,255,255,255))
ov=Image.new("RGBA",(W,H),(0,0,0,0))
for m,c,a in [(zone3,(0,180,0),100),(zone2,(255,210,0),150),(tumor_mask_proc,(220,20,60),150)]:
    if m.sum()==0: continue
    mask=Image.fromarray(m*255).convert("L")
    col=Image.new("RGBA",(W,H),c+(a,))
    ov.paste(col,(0,0),mask)
Image.alpha_composite(base,ov).save(OUT_DIR/"zones_overlay.png")

# TIL centroids
tils=[]
for feat in features:
    props=feat.get("properties",{}) or {}
    cname=props.get("classification",props.get("class"))
    if isinstance(cname,dict): cname=cname.get("name","Unknown")
    if "til" not in str(cname).lower(): continue
    geom=feat.get("geometry",{})
    coords=geom.get("coordinates") or []
    polys=[coords] if geom.get("type")=="Polygon" else coords if geom.get("type")=="MultiPolygon" else []
    for poly in polys:
        if not poly: continue
        cx,cy=centroid(poly[0])
        if cx: mx,my=to_mask_coord(cx,cy,downscale); tils.append((mx,my))

# border strips + waveform
inside_dt = distance_transform_edt(tumor_mask_proc==1)
outside_dt= distance_transform_edt(tumor_mask_proc==0)
signed=outside_dt; signed[tumor_mask_proc==1]=-inside_dt[tumor_mask_proc==1]

strips=np.zeros_like(signed,int)
for b in range(1,NUM_BINS_OUTSIDE+1):
    m=(signed>=(b-1)*BIN_WIDTH_PIXELS)&(signed<b*BIN_WIDTH_PIXELS)
    strips[m]=b
for b in range(1,NUM_BINS_INSIDE+1):
    m=(signed<=-(b-1)*BIN_WIDTH_PIXELS)&(signed>-b*BIN_WIDTH_PIXELS)
    strips[m]=-b

bin_ids=[strips[y,x] for x,y in tils if 0<=x<W and 0<=y<H]
bins=list(range(-NUM_BINS_INSIDE,0))+list(range(1,NUM_BINS_OUTSIDE+1))
centers=[]; densities=[]
for b in bins:
    area=(strips==b).sum()
    count=bin_ids.count(b)
    centers.append((-((abs(b)-0.5)*BIN_WIDTH_PIXELS) if b<0 else (b-0.5)*BIN_WIDTH_PIXELS)*downscale)
    densities.append(count/(area+1e-8))
dens=np.array(densities); dens=dens/dens.max() if dens.max()>0 else dens
plt.plot(centers,dens,marker='o'); plt.axvline(0,c='k',ls='--')
plt.xlabel("Distance (µm)"); plt.ylabel("Norm sTIL density")
plt.savefig(OUT_DIR/"til_waveform.png",dpi=150); plt.close()

print("Done. Outputs in:", OUT_DIR)